# Experimenting w tkinter

## Summary

It's possible to create fast, lightweight canvases using tkinter. By construction, canvases open in a separate window. In this folder we experiment with embedding this canvas in a cell, the way that iPyCanvas does.

tkinter canvases support styling options for drawing such as line color , marker width, and canvas background color. I suggest locking these options so the user writes with a relatively narrow (width<5) black marker on a white canvas to best suit the OCR.

Embedding tkinter is an ongoing process. tkinter operates in an independent window that serves as a canvas where drawing takes place. We must take this operation and insert it ina cell. ipycanvas works in this manner, but there is no interoperability between the tkinter class created to run the canvas and ipycanvas' method to populate a canvas inside a cell. See **Embedding in a cell** below to find the latest state of my thoughts on what could happen to make this work. 

## Simple canvas

The code below outputs a canvas in a separate window.


In [4]:
# from tkinter import * #originally wildcard import
from tkinter import Tk
from tkinter import Canvas
from tkinter import ttk
from tkinter import constants as con

def savePosn(event):
    global lastx, lasty
    lastx, lasty = event.x, event.y

def addLine(event):
    canvas.create_line((lastx, lasty, event.x, event.y))
    savePosn(event)

root = Tk()

root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

canvas = Canvas(root)
canvas.grid(column=0, row=0, sticky=(con.N, con.W, con.E ,con.S ))
canvas.bind("<Button-1>", savePosn)
canvas.bind("<B1-Motion>", addLine)

# print(type())

root.mainloop(0)

## Output 1:

The root object with the mainloop() method will open a window where drawing takes place.

<img src="./figures/tkinter_canvas_1.png" alt="Kitten" title="A cute kitten" width="350" height="100" /> 


## Another canvas

This canvas encapsulates some of the code above so global variables are not used.

In [5]:
# from tkinter import *
from tkinter import Tk
from tkinter import Canvas
from tkinter import ttk
from tkinter import constants as con

class Sketchpad(Canvas):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        self.bind("<Button-1>", self.save_posn)
        self.bind("<B1-Motion>", self.add_line)
        
    def save_posn(self, event):
        self.lastx, self.lasty = event.x, event.y

    def add_line(self, event):
        # default option
        #self.create_line((self.lastx, self.lasty, event.x, event.y))
        
        # some other ways to instantiate this line
        self.create_line((self.lastx, self.lasty, event.x, event.y),
                         fill="black", width=1)
        self.save_posn(event)

root = Tk()
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

# default
# sketch = Sketchpad(root)

# white background
sketch = Sketchpad(root,background="white")
sketch.grid(column=0, row=0, sticky=(con.N, con.W, con.E, con.S))

root.mainloop()

## Output 2:

With the default options.

<img src="./figures/tkinter_canvas_2.png" alt="" title="" width="350" height="100" /> 

## Output 3:

Playing with multiple options can be done like this:  

`self.create_line((self.lastx, self.lasty, event.x, event.y), fill="red", width=5)`

There are issues with wider paint brushes in that they tend to segment.  I think the color black with a width <5 is best so the OCR receives quality data.  Background color can alos play a role. Best set this to white so there is a higher contrast between writing (black) and canvas (white).

<img src="./figures/tkinter_canvas_3.png" alt="" title="" width="350" height="100" /> 

<img src="./figures/tkinter_canvas_4.png" alt="" title="" width="350" height="100" /> 

<img src="./figures/tkinter_canvas_5.png" alt="" title="" width="350" height="100" /> 

## Output 4:

Tkinter canvas with a white background.

`sketch = Sketchpad(root,background="white")`

<img src="./figures/tkinter_canvas_6.png" alt="" title="" width="350" height="100" /> 

## Embedding into a cell

Recall ipycanvas code, replicated here below.

tkinter embeds a canvas in a cell by using its ipywidget `HBox`.
Below are the imports for ipycanvas.

In [6]:
from ipywidgets import Image

from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox

from ipycanvas import RoughCanvas, hold_canvas

In [7]:
# the settings modify the canvas size
width = 2000
height = 1000

ipycanvas uses an object called RoughCanvas (type = <class 'ipycanvas.canvas.RoughCanvas'>) as the actual canvas widget the user draws on. 

Further down **HBox** (type = <class 'ipywidgets.widgets.widget_box.HBox'>) is applied to a tuple of widgets, which the user can interact with to style the drawing of the canvas. For now, the option below only admits the canvas and no other widgets. 

**HBox** is a cell container for the RoughCanvas. This what the user sees immediately below a cell to draw in. I'm thinking this can be modified to use tkinter's canvas instead, in lieu of finding a solution that exists for tkinter exclusively, which I don't exactly see.

In [23]:
canvas = RoughCanvas(width=width, height=height)

drawing = False
position = None
shape = []


def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]


def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    if not drawing:
        return

    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

        position = (x, y)

    shape.append(position)


def on_mouse_up(x, y):
    global drawing
    global position
    global shape

    drawing = False
    
    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)
#         canvas.fill_polygon(shape)

    shape = []


canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

canvas.stroke_style = '#749cb8'
canvas.line_width = 15.0

picker = ColorPicker(description='Color:', value='black')

link((picker, 'value'), (canvas, 'stroke_style'))
link((picker, 'value'), (canvas, 'fill_style'))

# diagnose: what type is this canvas that goes to HBox?
# print(type(canvas))

# HBox(canvas)

# diagnose: type of HBox
# print(type(HBox((canvas, canvas))))

# HBox that actually draws the widget being used.
HBox((canvas,))